In [0]:
import requests, json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, upper, current_timestamp, explode, regexp_replace, when, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType

In [0]:
url = "https://servicodados.ibge.gov.br/api/v2/censos/nomes/joao%7Cmaria%7Ccarlos%7Cfelipe%7Cguilherme%7Candressa"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Erro ao acessar a API: {response.status_code}")

In [0]:
res_schema = ArrayType(StructType([
        StructField("periodo", StringType(), True),
        StructField("frequencia", IntegerType(), True)
]))

schema = StructType([
    StructField("nome", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("localidade", StringType(), True),
    StructField("res", res_schema)
])

In [0]:
df_nome = spark.createDataFrame(data, schema=schema)

In [0]:
df_nome.printSchema()

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- localidade: string (nullable = true)
 |-- res: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- periodo: string (nullable = true)
 |    |    |-- frequencia: integer (nullable = true)



In [0]:
from pyspark.sql.functions import explode, col

df_nome = df_nome.withColumn("res_explodido", explode("res")) \
    .select(
        col("nome"),
        col("sexo"),
        col("localidade"),
        col("res_explodido.periodo").alias("periodo"),
        col("res_explodido.frequencia").alias("quantidade")
    )

df_nome.display()

nome,sexo,localidade,periodo,quantidade
ANDRESSA,null,BR,1930[,29
ANDRESSA,null,BR,"[1930,1940[",31
ANDRESSA,null,BR,"[1940,1950[",75
ANDRESSA,null,BR,"[1950,1960[",128
ANDRESSA,null,BR,"[1960,1970[",389
ANDRESSA,null,BR,"[1970,1980[",5380
ANDRESSA,null,BR,"[1980,1990[",33218
ANDRESSA,null,BR,"[1990,2000[",88232
ANDRESSA,null,BR,"[2000,2010[",58256
CARLOS,null,BR,1930[,4659


In [0]:
df_nome = df_nome.withColumn("periodo", regexp_replace(regexp_replace(col("periodo"), r"[\[\]]", ""), ",", "-"))

In [0]:
df_nome.display()

nome,sexo,localidade,periodo,quantidade
ANDRESSA,null,BR,1930,29
ANDRESSA,null,BR,1930-1940,31
ANDRESSA,null,BR,1940-1950,75
ANDRESSA,null,BR,1950-1960,128
ANDRESSA,null,BR,1960-1970,389
ANDRESSA,null,BR,1970-1980,5380
ANDRESSA,null,BR,1980-1990,33218
ANDRESSA,null,BR,1990-2000,88232
ANDRESSA,null,BR,2000-2010,58256
CARLOS,null,BR,1930,4659


In [0]:
df_nome = df_nome.withColumn("Sexo", when((col("Nome").isin("MARIA", "ANDRESSA")) & (col("Sexo").isNull()), "Feminino").otherwise(col("Sexo")))

In [0]:
df_nome = df_nome.withColumn("Sexo", when((col("Nome").isin("CARLOS", "JOAO", "FELIPE", "GUILHERME")) & (col("Sexo").isNull()), "Masculino").otherwise(col("Sexo")))

In [0]:
df_nome.display()

nome,Sexo,localidade,periodo,quantidade
ANDRESSA,Feminino,BR,1930,29
ANDRESSA,Feminino,BR,1930-1940,31
ANDRESSA,Feminino,BR,1940-1950,75
ANDRESSA,Feminino,BR,1950-1960,128
ANDRESSA,Feminino,BR,1960-1970,389
ANDRESSA,Feminino,BR,1970-1980,5380
ANDRESSA,Feminino,BR,1980-1990,33218
ANDRESSA,Feminino,BR,1990-2000,88232
ANDRESSA,Feminino,BR,2000-2010,58256
CARLOS,Masculino,BR,1930,4659


In [0]:
nascidos_ate_2000 = df_nome.filter(col("periodo").startswith("19") | col("periodo").like("%2000"))
nascidos_ate_2000.display()

nome,Sexo,localidade,periodo,quantidade
ANDRESSA,Feminino,BR,1930,29
ANDRESSA,Feminino,BR,1930-1940,31
ANDRESSA,Feminino,BR,1940-1950,75
ANDRESSA,Feminino,BR,1950-1960,128
ANDRESSA,Feminino,BR,1960-1970,389
ANDRESSA,Feminino,BR,1970-1980,5380
ANDRESSA,Feminino,BR,1980-1990,33218
ANDRESSA,Feminino,BR,1990-2000,88232
CARLOS,Masculino,BR,1930,4659
CARLOS,Masculino,BR,1930-1940,15620


In [0]:
agregado = nascidos_ate_2000.groupBy(col("periodo")).agg(sum("quantidade").alias("total")).orderBy(col("periodo"))
agregado.display()

periodo,total
1930,403094
1930-1940,910041
1940-1950,1801920
1950-1960,3034666
1960-1970,3203272
1970-1980,2184945
1980-1990,1656995
1990-2000,1611269


In [0]:
homens_ate_2000 = nascidos_ate_2000.filter(col("Sexo") == "Masculino").groupBy(col("periodo")).agg(sum("quantidade").alias("total")).orderBy(col("periodo"))
homens_ate_2000.display()

periodo,total
1930,66588
1930-1940,160957
1940-1950,314803
1950-1960,558056
1960-1970,707392
1970-1980,563546
1980-1990,705809
1990-2000,978741


In [0]:
mulheres_ate_2000 = nascidos_ate_2000.filter(col("Sexo") == "Feminino").groupBy("periodo").agg(sum("quantidade").alias("Total")).orderBy(col("periodo"))
mulheres_ate_2000.display()

periodo,Total
1930,336506
1930-1940,749084
1940-1950,1487117
1950-1960,2476610
1960-1970,2495880
1970-1980,1621399
1980-1990,951186
1990-2000,632528


In [0]:
delta_path = "/Volumes/workspace/default/dados/nome/"

df_nome.write.format("delta").save(delta_path)

print(f"Dados salvos com sucesso em: {delta_path}")




Dados salvos com sucesso em: /Volumes/workspace/default/dados/nome/
